In [37]:
import gensim
import sqlite3
from operator import itemgetter

In [21]:
class WordSplitter(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
        with open(self.filename) as fin:
            for line in fin:
                yield line.split()

model = gensim.models.Word2Vec(model_input, min_count=4)

In [56]:
model.save(open('zoo/15/songs.word2vec', 'wb'))

MemoryError: 

In [71]:
conn = sqlite3.connect('data/songs.db')
def find_song(song_name, limit=10):
    c = conn.cursor()
    c.execute("SELECT * FROM songs WHERE UPPER(name) LIKE '%" + song_name + "%'")
    res = sorted((x + (model.wv.vocab[x[0]].count,) 
                  for x in c.fetchall() if x[0] in model.wv.vocab),
                 key=itemgetter(-1), reverse=True)
    return [*res][:limit]

for t in find_song('the eye of the tiger'):
    print(*t)

2ZqGzZWWZXEyPxJy6N9QhG The eye of the tiger - version éditée Olivier Bernet - Arnaud Aguergaray - Stéphane Garin - Stéphane Goueytes - Maitane Sebastian - Chiara Mastroianni 39
4rr0ol3zvLiEBmep7HaHtx The Eye Of The Tiger Survivor 37
0R85QWa6KRzB8p44XXE7ky The Eye of the Tiger Gloria Gaynor 29
3GxdO4rTwVfRvLRIZFXJVu The Eye of the Tiger Gloria Gaynor 19
1W602jfZkdAsbabmJEYfFi The Eye of the Tiger Gloria Gaynor 5
6g197iis9V2HP7gvc5ZpGy I Got the Eye of the Tiger Circus Roar 5
00VQxzTLqwqBBE0BuCVeer The Eye Of The Tiger Gloria Gaynor 5
28FwycRDU81YOiGgIcxcPq The Eye of the Tiger Gloria Gaynor 5
62UagxK6LuPbqUmlygGjcU It's the Eye of the Tiger Be Cult 4
6lUHKc9qrIHvkknXIrBq6d The Eye Of The Tiger Survivor 4


In [72]:
def suggest_songs(song_id):
    c = conn.cursor()
    similar = dict(model.most_similar([song_id]))
    song_ids = ', '.join(("'%s'" % x) for x in similar.keys())
    c.execute("SELECT * FROM songs WHERE id in (%s)" % song_ids)
    res = sorted((rec + (similar[rec[0]],) for rec in c.fetchall()), 
                 key=itemgetter(-1),
                 reverse=True)
    return [*res]


for t in suggest_songs('4rr0ol3zvLiEBmep7HaHtx'):
    print(*t)

6lQMrYnRqv3ILwvpmHWXjT Girls Just Wanna Have Fun Cyndi Lauper 0.9735351204872131
3f2UsY38rm3QOkxnzyyAMb Enola Gay - 2003 Remastered Version Orchestral Manoeuvres In The Dark 0.9719518423080444
6nzQPViPJEjysZBybcIVKt You're My Heart, You're My Soul Modern Talking 0.9589041471481323
1qHuRij9fsG8duSmeupC27 Gold - 2003 Remastered Version Spandau Ballet 0.9566971659660339
6r8rLHn2H5KEss31OexCz8 Dolce Vita Ryan Paris 0.9553133249282837
2nt2B4wZsOP2VsxyzpjOwQ Karma Chameleon - 2002 Remastered Version Culture Club 0.9531201720237732
2u0j4TA6K7zNMAstX48oXK Bette Davis Eyes Kim Carnes 0.9499865770339966
7p9Tsoczgmvd50wwnz9kai Walking On Sunshine Katrina & The Waves 0.9481900930404663
0RY4siAV10AgtLPIn37sBC Maneater Daryl Hall & John Oates 0.9481032490730286
62sd69ls8HdPUlfD75axcN Don't You Want Me - 2002 Remastered Version The Human League 0.9471924901008606


/home/douwe/proj/notebooks/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
x = model.wv.vocab['2yqVlvXgZ51ynGqwfYADx4']
x.count

53